In [1]:
# Billing for four builings 
import numpy as np

# System Parameters
# Loads (kWh)
L = np.array([100, 50, 150, 160])

# Solar Production (kWh, reduced to ~10% capacity for shortfall)
S = np.array([200, 60, 20, 20])

# Number of buildings
n_buildings = 4

# Initialize transfer matrix E (E[i,j] is energy from building i to j)
E = np.zeros((n_buildings, n_buildings))

# Step 1: Calculate Surplus and Deficit
surplus_deficit = S - L
surplus = np.zeros(n_buildings)
deficit = np.zeros(n_buildings)

for i in range(n_buildings):
    if surplus_deficit[i] > 0:
        surplus[i] = surplus_deficit[i]
    else:
        deficit[i] = -surplus_deficit[i]

total_surplus = np.sum(surplus)  # 110 kWh
total_deficit = np.sum(deficit)   # 270 kWh

# Step 2: Pre-transfer MDB Readings
M_pre = np.zeros(n_buildings)
for i in range(n_buildings):
    if surplus_deficit[i] > 0:
        M_pre[i] = -surplus_deficit[i]  # Negative for exporters
    else:
        M_pre[i] = -surplus_deficit[i]  # Positive for importers (L_i - S_i)

# Step 3: Allocate Transfers Using Billing Equation
# Deficit shares for importing buildings (Buildings 3 and 4)
deficit_shares = np.zeros(n_buildings)
for i in range(n_buildings):
    if deficit[i] > 0:
        deficit_shares[i] = deficit[i] / total_deficit

# Surplus shares for exporting buildings (Buildings 1 and 2)
surplus_shares = np.zeros(n_buildings)
for i in range(n_buildings):
    if surplus[i] > 0:
        surplus_shares[i] = surplus[i] / total_surplus

# Calculate transfers E[i,j]
for i in range(n_buildings):
    if M_pre[i] < 0:  # Exporting building
        surplus_i = -M_pre[i]  # Surplus to export
        for j in range(n_buildings):
            if deficit[j] > 0:  # Importing building
                E[i, j] = deficit_shares[j] * surplus_i
                # Round to nearest integer for simplicity
                E[i, j] = round(E[i, j])

# Step 4: Calculate MDB Meter Readings
M = np.zeros(n_buildings)
for i in range(n_buildings):
    # Energy balance: L_i = S_i + M_i + sum(E_k->i) - sum(E_i->j)
    received = np.sum(E[:, i])  # Sum of E[k,i] for k != i
    exported = np.sum(E[i, :])  # Sum of E[i,j] for j != i
    M[i] = L[i] - S[i] - received + exported

# Step 5: Grid Import
M5 = np.sum(M)  # M5 = M1 + M2 + M3 + M4

# Step 6: Verify Billing Equation with Final M_j
# Recalculate transfers using final M_j for importing buildings
total_import = np.sum(M[M > 0])  # Sum of positive M_j (importing buildings)
if total_import > 0:
    import_shares = np.zeros(n_buildings)
    for j in range(n_buildings):
        if M[j] > 0:
            import_shares[j] = M[j] / total_import

    # Recompute transfers
    E_final = np.zeros((n_buildings, n_buildings))
    for i in range(n_buildings):
        if M_pre[i] < 0:  # Exporting building
            surplus_i = -M_pre[i]
            for j in range(n_buildings):
                if M[j] > 0:  # Importing building
                    E_final[i, j] = import_shares[j] * surplus_i
                    E_final[i, j] = round(E_final[i, j])

else:
    E_final = E  # If no grid import, use initial transfers

# Step 7: Output Results
print("System Parameters:")
print(f"Loads (kWh): {L}")
print(f"Solar Production (kWh): {S}")
print("\nSurplus/Deficit (kWh):")
for i in range(n_buildings):
    print(f"Building {i+1}: {surplus_deficit[i]:.0f}")
print(f"Total Surplus: {total_surplus:.0f} kWh")
print(f"Total Deficit: {total_deficit:.0f} kWh")
print(f"Shortfall (Grid Import): {total_deficit - total_surplus:.0f} kWh")

print("\nPre-transfer MDB Readings (kWh):")
for i in range(n_buildings):
    print(f"Building {i+1}: {M_pre[i]:.0f}")

print("\nEnergy Transfers (kWh):")
for i in range(n_buildings):
    for j in range(n_buildings):
        if E_final[i, j] > 0:
            print(f"E_{i+1}->{j+1}: {E_final[i, j]:.0f}")

print("\nMDB Meter Readings (kWh):")
for i in range(n_buildings):
    print(f"M_{i+1}: {M[i]:.0f}")
print(f"Grid (M_5): {M5:.0f} kWh")

print("\nBilling Outcome:")
print("Inter-building Billing:")
for j in range(n_buildings):
    if deficit[j] > 0:
        print(f"Building {j+1}:")
        for i in range(n_buildings):
            if E_final[i, j] > 0:
                print(f"  {E_final[i, j]:.0f} kWh from Building {i+1}")
for i in range(n_buildings):
    if surplus[i] > 0:
        total_exported = np.sum(E_final[i, :])
        print(f"Building {i+1}: Credited for {total_exported:.0f} kWh")

print("\nGrid Billing:")
for i in range(n_buildings):
    if M[i] > 0:
        print(f"Building {i+1}: {M[i]:.0f} kWh")
    else:
        print(f"Building {i+1}: 0 kWh")
print(f"Total Grid Import: {M5:.0f} kWh")

System Parameters:
Loads (kWh): [100  50 150 160]
Solar Production (kWh): [200  60  20  20]

Surplus/Deficit (kWh):
Building 1: 100
Building 2: 10
Building 3: -130
Building 4: -140
Total Surplus: 110 kWh
Total Deficit: 270 kWh
Shortfall (Grid Import): 160 kWh

Pre-transfer MDB Readings (kWh):
Building 1: -100
Building 2: -10
Building 3: 130
Building 4: 140

Energy Transfers (kWh):
E_1->3: 48
E_1->4: 52
E_2->3: 5
E_2->4: 5

MDB Meter Readings (kWh):
M_1: 0
M_2: 0
M_3: 77
M_4: 83
Grid (M_5): 160 kWh

Billing Outcome:
Inter-building Billing:
Building 3:
  48 kWh from Building 1
  5 kWh from Building 2
Building 4:
  52 kWh from Building 1
  5 kWh from Building 2
Building 1: Credited for 100 kWh
Building 2: Credited for 10 kWh

Grid Billing:
Building 1: 0 kWh
Building 2: 0 kWh
Building 3: 77 kWh
Building 4: 83 kWh
Total Grid Import: 160 kWh
